In [57]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
import string
import random
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pickle
import os

In [58]:
demo_df = pd.read_csv(r"test.csv")

In [59]:
demo_df

,Unnamed: 0,PassengerId,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Family_size,Title_1,Title_2,Title_3,Title_4,Emb_1,Emb_2,Emb_3
0,791,792,1,0.2000,0.050749,0,1,0,0.0,1,0,0,0,0,0,1
1,792,793,0,0.3500,0.135753,0,0,1,1.0,0,0,0,1,0,0,1
2,793,794,1,0.3500,0.059914,1,0,0,0.0,1,0,0,0,1,0,0
3,794,795,1,0.3125,0.015412,0,0,1,0.0,1,0,0,0,0,0,1
4,795,796,1,0.4875,0.025374,0,1,0,0.0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,886,887,1,0.3375,0.025374,0,1,0,0.0,1,0,0,0,0,0,1
96,887,888,0,0.2375,0.058556,1,0,0,0.0,0,0,0,1,0,0,1
97,888,889,0,0.3500,0.045771,0,0,1,0.3,0,0,0,1,0,0,1
98,889,890,1,0.3250,0.058556,1,0,0,0.0,1,0,0,0,1,0,0


In [78]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/automl-bigdata-7c2859c8477a.json' 

In [79]:
from google.cloud import storage
def write_read(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename('/content/Dummy Data HSS.csv')

In [80]:
from google.cloud import storage


def write_read(bucket_name, blob_name):
    """Write and read a blob from GCS using file-like IO"""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your new GCS object
    # blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Mode can be specified as wb/rb for bytes mode.
    # See: https://docs.python.org/3/library/io.html
    with blob.open("w") as f:
        f.write("Hello world")

    with blob.open("r") as f:
        print(f.read())


In [81]:
def impute(data):
    #checking missing values
    percent_missing = data.isnull().sum() * 100 / data.shape[0]
    #dropping columns if missing percentage is more than 30
    for i in range(len(data.columns)):
        if percent_missing[i] >30:
            data.drop(data.columns[i],axis=1,inplace=True)
    #getting numerical and categorical variables
    numerical_columns = [x for x in data.columns if data[x].dtype != 'object']
    data_num = data[numerical_columns]
    
    cat_columns = [x for x in data.columns if x not in numerical_columns]
    data_cat = data[cat_columns]
    
    #Imputing using KNN Imputer for numerical columns
    imputer = KNNImputer(n_neighbors=2)
    imputed_num = imputer.fit_transform(data_num)
    imputed_num = pd.DataFrame(imputed_num)
    imputed_num.columns=data_num.columns
    
    # most frequent imputation for categorical columns
    data_cat_imputed = data_cat.apply(lambda x: x.fillna(x.value_counts().index[0]))
    
    #concat the imputed dfs
    imputed_data = pd.concat([imputed_num, data_cat_imputed], axis=1)
    
    #return imputed_data
    return imputed_data

In [82]:
ll = impute(demo_df)

In [83]:
def normalize_and_encode(imputed_data):
    #normalizing numerical columns using robustscalar
    numerical_columns  = [x for x in imputed_data.columns if imputed_data[x].dtype in ['int64', 'float64']]
    scalar = RobustScaler(quantile_range=(25,75))
    scaled = scalar.fit_transform(imputed_data[numerical_columns])
    scaled = pd.DataFrame(scaled)
    scaled.columns = imputed_data[numerical_columns].columns
    
    #dropping cat columns with more than 10 categories
    cat_cols = [x for x in imputed_data.columns if x not in numerical_columns]
    cat_cols_to_drop = []
    for col in cat_cols:
        if imputed_data[col].value_counts().count()>10:
            cat_cols_to_drop.append(col)
    data_for_enc = imputed_data.drop(numerical_columns,axis=1)
    data_for_enc.drop(cat_cols_to_drop,axis=1,inplace=True)

    #encoding categorical varialbles
    try:
        enc_data= pd.get_dummies(data_for_enc, columns=data_for_enc.columns)
        encoded_data = pd.concat([scaled, enc_data], axis=1)
    except:
      encoded_data = scaled.copy()

    return encoded_data

In [84]:
test = normalize_and_encode(ll)

In [90]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/automl-bigdata-7c2859c8477a.json' 
def cloud_read(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    pickle_in = blob.download_as_string()
    my_dictionary = pickle.loads(pickle_in)
    #pickled_model = pickle.load(open(data, 'rb'))
    print(f'Pulled down file from bucket {bucket_name}, file name: {blob_name}')
    return my_dictionary

In [91]:
pickled_model=cloud_read('automl-bigdataarch','classification_models/model.pkl')

Pulled down file from bucket automl-bigdataarch, file name: classification_models/model.pkl


In [ ]:
pickled_model=cloud_read('automl-bigdataarch','regression_models/model.pkl')

In [92]:
pred_file = pickled_model.predict(test)
pd.DataFrame({"Predictions":pred_file}).to_csv("pred_file.csv",index=False)

In [94]:
def cloud_pred(bucket_name, blob_name, pred_file):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(pred_file)

In [96]:
cloud_pred('automl-bigdataarch', 'predictions/pred_file', 'pred_file.csv')